In [50]:
import h2o
from h2o.automl import H2OAutoML

In [51]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,1 hour 28 mins
H2O_cluster_timezone:,Europe/Moscow
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.0.4
H2O_cluster_version_age:,6 months and 5 days !!!
H2O_cluster_name:,H2O_from_python_leor_finkelberg_miozzy
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.772 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


In [52]:
data = h2o.import_file("./data/Dataset.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [57]:
train, val = data.split_frame([0.85,])

In [58]:
features_lst = train.columns
target = 'output'
features_lst.remove(target)

In [59]:
aml = H2OAutoML(max_models=10, seed=42)
aml.train(
    x=features_lst, y=target,
    training_frame=train,
    validation_frame=val
)

AutoML progress: |
00:13:04.106: User specified a validation frame with cross-validation still enabled. Please note that the models will still be validated using cross-validation only, the validation frame will be used to provide purely informative validation metrics on the trained models.

████████████████████████████████████████████████████████| 100%


In [61]:
lb = aml.leaderboard
lb.head(rows=lb.nrows)

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
StackedEnsemble_AllModels_AutoML_20201207_001304,463.281,21.524,463.281,4.15506,nan
StackedEnsemble_BestOfFamily_AutoML_20201207_001304,466.901,21.6079,466.901,4.18861,nan
GBM_3_AutoML_20201207_001304,471.139,21.7057,471.139,4.27358,nan
GBM_2_AutoML_20201207_001304,478.735,21.88,478.735,4.3144,nan
GBM_4_AutoML_20201207_001304,479.889,21.9064,479.889,4.31001,nan
GBM_1_AutoML_20201207_001304,484.584,22.0133,484.584,4.2513,nan
DRF_1_AutoML_20201207_001304,497.093,22.2956,497.093,4.27742,0.572677
XGBoost_3_AutoML_20201207_001304,502.587,22.4184,502.587,4.49027,nan
GBM_5_AutoML_20201207_001304,515.335,22.701,515.335,4.61913,nan
XGBoost_1_AutoML_20201207_001304,531.357,23.0512,531.357,4.79753,nan


In [62]:
aml_leader = aml.leader
preds = aml_leader.predict(test)

stackedensemble prediction progress: |████████████████████████████████████| 100%


In [68]:
r2_score_valid = aml_leader.r2(valid=True); print(f"R^2 = {r2_score_valid:.3f}")

R^2 = 0.776


In [70]:
h2o.download_model(aml_leader, path=".")

'/Users/leor.finkelberg/Jupyter_projects/OTUS_tasks/linearRegression/StackedEnsemble_AllModels_AutoML_20201207_001304'